# Work in Progress!!

## Exploring the `fhirclient` package: Creating a transaction Bundle containing Patient, Speciment, and Observation Resources

Here we will create a Patient resource, a Specimen from that Patient, and an Observation from that Specimen. All of these will be placed into a FHIR transaction Bundle and uploaded to a FHIR server.

Note: some of this material is from `makePatient.ipynb` and `makeSpecimen.ipynb` notebooks.

* install with 
    * `pip install fhirclient`
    * if you are using `pipenv` to manage your environments, then you can instead run `pipenv install fhirclient`
    * note: I modified `fhirclient/models/fhirabstractresource.py` so I can get the full response headers, and also have transaction bundles POST to the base url (see below), so copied the [`fhirclient`](https://github.com/smart-on-fhir/client-py/tree/master/fhirclient) package into my repo and edited it there. Doing this, I don't need to do the `pip install fhirclient`.
* versions
    * 3.2 (current at time of writing) is for STU3
    * 1.0.3 is for DSTU2
* github
    * https://github.com/smart-on-fhir/client-py
* Documentation
    * http://docs.smarthealthit.org/client-py/



---
## Creating a Patient

    
There is an example in the fhirclient documentation that shows how to create a Patient resource...

In [1]:
import fhirclient.models.patient as p
patient = p.Patient()

But the docs are sparse at describing the methods and attributes that are in it, at least for newbies. 

### Using `help()`
We can use `help()` to find out more about the `patient` object.

In [2]:
help(patient)

Help on Patient in module fhirclient.models.patient object:

class Patient(fhirclient.models.domainresource.DomainResource)
 |  Patient(jsondict=None, strict=True)
 |  
 |  Information about an individual or animal receiving health care services.
 |  
 |  Demographics and other administrative information about an individual or
 |  animal receiving care or other health-related services.
 |  
 |  Method resolution order:
 |      Patient
 |      fhirclient.models.domainresource.DomainResource
 |      fhirclient.models.resource.Resource
 |      fhirclient.models.fhirabstractresource.FHIRAbstractResource
 |      fhirclient.models.fhirabstractbase.FHIRAbstractBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, jsondict=None, strict=True)
 |      Initialize all valid properties.
 |      
 |      :raises: FHIRValidationError on validation errors, unless strict is False
 |      :param dict jsondict: A JSON dictionary to use for initialization
 |      :param bool 

### The `.elementProperties()` method looks interesting

```
|  elementProperties(self)
|    Returns a list of tuples, one tuple for each property that should
|    be serialized, as: ("name", "json_name", type, is_list, "of_many", not_optional)
```

In [3]:
patient.elementProperties()

[('id', 'id', str, False, None, False),
 ('implicitRules', 'implicitRules', str, False, None, False),
 ('language', 'language', str, False, None, False),
 ('meta', 'meta', fhirclient.models.meta.Meta, False, None, False),
 ('contained',
  'contained',
  fhirclient.models.resource.Resource,
  True,
  None,
  False),
 ('extension',
  'extension',
  fhirclient.models.extension.Extension,
  True,
  None,
  False),
 ('modifierExtension',
  'modifierExtension',
  fhirclient.models.extension.Extension,
  True,
  None,
  False),
 ('text', 'text', fhirclient.models.narrative.Narrative, False, None, False),
 ('active', 'active', bool, False, None, False),
 ('address', 'address', fhirclient.models.address.Address, True, None, False),
 ('animal',
  'animal',
  fhirclient.models.patient.PatientAnimal,
  False,
  None,
  False),
 ('birthDate',
  'birthDate',
  fhirclient.models.fhirdate.FHIRDate,
  False,
  None,
  False),
 ('communication',
  'communication',
  fhirclient.models.patient.PatientComm

Yep, we see a list of tuples. 

Let's make it a little easier to read.

In [4]:
for eproperty in patient.elementProperties():
    print(eproperty)

('id', 'id', <class 'str'>, False, None, False)
('implicitRules', 'implicitRules', <class 'str'>, False, None, False)
('language', 'language', <class 'str'>, False, None, False)
('meta', 'meta', <class 'fhirclient.models.meta.Meta'>, False, None, False)
('contained', 'contained', <class 'fhirclient.models.resource.Resource'>, True, None, False)
('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('modifierExtension', 'modifierExtension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('text', 'text', <class 'fhirclient.models.narrative.Narrative'>, False, None, False)
('active', 'active', <class 'bool'>, False, None, False)
('address', 'address', <class 'fhirclient.models.address.Address'>, True, None, False)
('animal', 'animal', <class 'fhirclient.models.patient.PatientAnimal'>, False, None, False)
('birthDate', 'birthDate', <class 'fhirclient.models.fhirdate.FHIRDate'>, False, None, False)
('communication', 'communicatio

Each tuple has six members. From the `help(patient)` result, we see what each member is:
```
|  elementProperties(self)
|    Returns a list of tuples, one tuple for each property that should
|    be serialized, as: ("name", "json_name", type, is_list, "of_many", not_optional)
```
The first three items in a tuple are easy to figure out. The last three required looking at the FHIR [Patient](http://build.fhir.org/patient.html) resource specification to confirm suspicions.
* `is_list` = Boolean that indicates if more than one member is allowed and therefore should be in a `list`, e.g., cardinality of `0..*`
* `of_many` = choice of data types (pick one), with prefix indicated, e.g, `deceasedBoolean` or `deceasedDateTime`
* `not_optional` = required, e.g., cardinality of `1..*`

How many of these `patient` properties are required?

In [5]:
required = []
for eproperty in patient.elementProperties():
    if eproperty[5] == True:
        required.append(eproperty)
len(required)

0

In [ ]:
from collections import namedtuple
namedProperties = namedtuple("name", "json_name", "type", "is_list", "of_many", "not_optional")
e = namedProperties(*)

Nothing is actually required for this resource. Everything is optional.

How many can have more than one and need to be captured in a `list`?

In [6]:
islist = []
for eproperty in patient.elementProperties():
    if eproperty[3] == True:
        islist.append(eproperty)
len(islist)

12

What are they?

In [7]:
for eproperty in islist:
    print(eproperty)

('contained', 'contained', <class 'fhirclient.models.resource.Resource'>, True, None, False)
('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('modifierExtension', 'modifierExtension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('address', 'address', <class 'fhirclient.models.address.Address'>, True, None, False)
('communication', 'communication', <class 'fhirclient.models.patient.PatientCommunication'>, True, None, False)
('contact', 'contact', <class 'fhirclient.models.patient.PatientContact'>, True, None, False)
('generalPractitioner', 'generalPractitioner', <class 'fhirclient.models.fhirreference.FHIRReference'>, True, None, False)
('identifier', 'identifier', <class 'fhirclient.models.identifier.Identifier'>, True, None, False)
('link', 'link', <class 'fhirclient.models.patient.PatientLink'>, True, None, False)
('name', 'name', <class 'fhirclient.models.humanname.HumanName'>, True, None, False)
('photo', 'photo

Those 12 properties must be captured in a `list`.

Which, if any, of the patient properties are `of_many`?

In [8]:
for eproperty in patient.elementProperties():
    if eproperty[4] != None:
        print(eproperty)

('deceasedBoolean', 'deceasedBoolean', <class 'bool'>, False, 'deceased', False)
('deceasedDateTime', 'deceasedDateTime', <class 'fhirclient.models.fhirdate.FHIRDate'>, False, 'deceased', False)
('multipleBirthBoolean', 'multipleBirthBoolean', <class 'bool'>, False, 'multipleBirth', False)
('multipleBirthInteger', 'multipleBirthInteger', <class 'int'>, False, 'multipleBirth', False)


<div class="alert alert-block alert-info">A word about choice of data types. From http://build.fhir.org/formats.html#choice,
<p>
<i>Elements that have a choice of data type cannot repeat - they must have a maximum cardinality of 1. When constructing an instance of an element with a choice of types, the authoring system must create a single element with a data type chosen from among the list of permitted data types.</i>
</div>

### The `.as_json()` method
Another method for the patient object is `.as_json()` which was inherited from <br>
`fhirclient.models.fhirabstractresource.FHIRAbstractResource`<p>
from `help(patient)`
```
 |   as_json(self)
 |      Serializes to JSON by inspecting `elementProperties()` and creating
 |      a JSON dictionary of all registered properties. Checks:
 |      
 |      - whether required properties are not None (and lists not empty)
 |      - whether not-None properties are of the correct type
 |      
 |      :raises: FHIRValidationError if properties have the wrong type or if
 |          required properties are empty
 |      :returns: A validated dict object that can be JSON serialized
```
    
Let's try it.

In [9]:
patient.as_json()

{'resourceType': 'Patient'}

When patient was created, it automatically filled in the `resourceType`, which interestingly isn't one of the properties returned by `patient.elementProperties`

We can pretty print this with `json.dumps()`

In [10]:
import json
print(json.dumps(patient.as_json(), indent=4))

{
    "resourceType": "Patient"
}


### Add a name to Patient

We see from `patient.elementsProperties()` that `name` has a type of `<class 'fhirclient.models.humanname.HumanName'>`

We'll create a `HumanName` object to hold this, and find out the properties it has.

In [11]:
import fhirclient.models.humanname as hn
name = hn.HumanName()
for eproperty in name.elementProperties():
    print(eproperty)

('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('id', 'id', <class 'str'>, False, None, False)
('family', 'family', <class 'str'>, False, None, False)
('given', 'given', <class 'str'>, True, None, False)
('period', 'period', <class 'fhirclient.models.period.Period'>, False, None, False)
('prefix', 'prefix', <class 'str'>, True, None, False)
('suffix', 'suffix', <class 'str'>, True, None, False)
('text', 'text', <class 'str'>, False, None, False)
('use', 'use', <class 'str'>, False, None, False)


#### Let's build a name
Since `given` has `is_list` set to `True`, we need to put it in a list or we'll get a `FHIRValidationError`. This is so we can have additional names as part of the full name (e.g., middle names). Since we can, we'll add a middle name. 

In [12]:
name.family = 'Storm'
name.given = ['Jonathon', 'Lowell']
name.text = 'Jonathon Lowell Storm'

We can now define how the name is used. I haven't figured out if the valueset for `use` can be found in the models, but reading the specification for [`HumanName`](http://hl7.org/implement/standards/fhir/datatypes.html#HumanName), I see that `use` must be from http://hl7.org/implement/standards/fhir/valueset-name-use.html. We'll make it for `official` use. If we mistype this, a `FHIRValidationError` won't pop up here, but the FHIR server might when we POST it.

In [13]:
name.use = 'official'
print(json.dumps(name.as_json(), indent=4))

{
    "family": "Storm",
    "given": [
        "Jonathon",
        "Lowell"
    ],
    "text": "Jonathon Lowell Storm",
    "use": "official"
}


Now add `name` to `patient`. Again, since `name` has `is_list` set to true, we need to put it in a list.

In [14]:
patient.name = [name]
print(json.dumps(patient.as_json(), indent=4))

{
    "name": [
        {
            "family": "Storm",
            "given": [
                "Jonathon",
                "Lowell"
            ],
            "text": "Jonathon Lowell Storm",
            "use": "official"
        }
    ],
    "resourceType": "Patient"
}


Since we can have more than one name, let's add a couple of nicknames.

In [15]:
name2 = hn.HumanName()
name2.given = ['Johnny']
name2.family = 'Storm'
name2.use = 'nickname'
name3 = hn.HumanName()
name3.text = 'Human Torch'
name3.use = 'nickname'
patient.name.extend([name2, name3])

Now let's see what the final Patient resource instance looks like.

In [16]:
print(json.dumps(patient.as_json(), indent=4))

{
    "name": [
        {
            "family": "Storm",
            "given": [
                "Jonathon",
                "Lowell"
            ],
            "text": "Jonathon Lowell Storm",
            "use": "official"
        },
        {
            "family": "Storm",
            "given": [
                "Johnny"
            ],
            "use": "nickname"
        },
        {
            "text": "Human Torch",
            "use": "nickname"
        }
    ],
    "resourceType": "Patient"
}


When we put this Patient into a Bundle resource, we will need to tag it with a UUID that other resources in the bundle can reference. We can generate it using the `uuid` package, and just create a new element to hold in in `patient`.

In [17]:
import uuid
patient.uuid = uuid.uuid4().urn
print(patient.uuid)

urn:uuid:d3bc8ddd-ba12-4646-9757-01ca4b2c2192


One thing I discovered is that even though I can add an attribute to a resource object, it won't affect the behaviour of the methods. For example, I can see `uuid` when I do a `patient.__dict__`, but it's not listed among the `.elementsProperties()` method.

In [18]:
import pprint as pp
pp.pprint(patient.__dict__)

{'_owner': None,
 '_resolved': None,
 '_server': None,
 'active': None,
 'address': None,
 'animal': None,
 'birthDate': None,
 'communication': None,
 'contact': None,
 'contained': None,
 'deceasedBoolean': None,
 'deceasedDateTime': None,
 'extension': None,
 'gender': None,
 'generalPractitioner': None,
 'id': None,
 'identifier': None,
 'implicitRules': None,
 'language': None,
 'link': None,
 'managingOrganization': None,
 'maritalStatus': None,
 'meta': None,
 'modifierExtension': None,
 'multipleBirthBoolean': None,
 'multipleBirthInteger': None,
 'name': [<fhirclient.models.humanname.HumanName object at 0x111361278>,
 'photo': None,
 'telecom': None,
 'text': None,
 'uuid': 'urn:uuid:d3bc8ddd-ba12-4646-9757-01ca4b2c2192'}


In [19]:
print(json.dumps(patient.as_json(), indent=4))

{
    "name": [
        {
            "family": "Storm",
            "given": [
                "Jonathon",
                "Lowell"
            ],
            "text": "Jonathon Lowell Storm",
            "use": "official"
        },
        {
            "family": "Storm",
            "given": [
                "Johnny"
            ],
            "use": "nickname"
        },
        {
            "text": "Human Torch",
            "use": "nickname"
        }
    ],
    "resourceType": "Patient"
}


---

### Building a Specimen Resource

We will build a Specimen resource using the Patient we just created. We'll include information that the specimen was from a buccal swab.

import additional packages we need...

In [20]:
import fhirclient.models.specimen as s
import fhirclient.models.fhirreference as fr
import fhirclient.models.codeableconcept as cc
import fhirclient.models.coding as co

Create a Specimen object and take a look at its' `.elementProperties()`.

In [21]:
mySpecimen = s.Specimen()
for eproperty in mySpecimen.elementProperties():
    print(eproperty)

('id', 'id', <class 'str'>, False, None, False)
('implicitRules', 'implicitRules', <class 'str'>, False, None, False)
('language', 'language', <class 'str'>, False, None, False)
('meta', 'meta', <class 'fhirclient.models.meta.Meta'>, False, None, False)
('contained', 'contained', <class 'fhirclient.models.resource.Resource'>, True, None, False)
('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('modifierExtension', 'modifierExtension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('text', 'text', <class 'fhirclient.models.narrative.Narrative'>, False, None, False)
('accessionIdentifier', 'accessionIdentifier', <class 'fhirclient.models.identifier.Identifier'>, False, None, False)
('collection', 'collection', <class 'fhirclient.models.specimen.SpecimenCollection'>, False, None, False)
('container', 'container', <class 'fhirclient.models.specimen.SpecimenContainer'>, True, None, False)
('identifier', 'identifier', <class

Which, if any, are required? We'll look at the `not_optional` boolean which is in the last position of the tuple.

Since we'll do this again for other objects, we'll create a little function to determine this.

In [22]:
def whatsRequired(fhirObject):
    required = False
    for eproperty in fhirObject.elementProperties():
        if eproperty[5] == True:
            print(eproperty)
            required = True
    if not required:
        print("Nothing is required")

In [23]:
whatsRequired(mySpecimen)

('subject', 'subject', <class 'fhirclient.models.fhirreference.FHIRReference'>, False, None, True)


Only `subject` is required. Our Subject will be the Patient we created in the previous notebook. We'll use the `uuid` of the Patient that was posted to the same server.
We see from `.elementProperties()` that `mySpecimen.subject` is a type of `fhirclient.models.fhirreference.FHIRReference`, so we'll build this.

First, create the `FHIRReference` object.</p>

In [24]:
mySubject = fr.FHIRReference()
for eproperty in mySubject.elementProperties():
    print(eproperty)

('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('id', 'id', <class 'str'>, False, None, False)
('display', 'display', <class 'str'>, False, None, False)
('identifier', 'identifier', <class 'fhirclient.models.identifier.Identifier'>, False, None, False)
('reference', 'reference', <class 'str'>, False, None, False)


Which of these is required?

In [25]:
whatsRequired(mySubject)

Nothing is required


None of the elements inside of `subject` are required. We'll use the `reference` and point to the Patient we created, and then take a look to see how `mySubject` looks so far.

In [26]:
mySubject.reference = patient.uuid
mySpecimen.subject = mySubject
import json
print(json.dumps(mySpecimen.as_json(), indent=4))

{
    "subject": {
        "reference": "urn:uuid:d3bc8ddd-ba12-4646-9757-01ca4b2c2192"
    },
    "resourceType": "Specimen"
}


Now we'll add some collection information to this Specimen. 

In [27]:
myCollection = s.SpecimenCollection()
for eproperty in myCollection.elementProperties():    
    print(eproperty)

('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('id', 'id', <class 'str'>, False, None, False)
('modifierExtension', 'modifierExtension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('bodySite', 'bodySite', <class 'fhirclient.models.codeableconcept.CodeableConcept'>, False, None, False)
('collectedDateTime', 'collectedDateTime', <class 'fhirclient.models.fhirdate.FHIRDate'>, False, 'collected', False)
('collectedPeriod', 'collectedPeriod', <class 'fhirclient.models.period.Period'>, False, 'collected', False)
('collector', 'collector', <class 'fhirclient.models.fhirreference.FHIRReference'>, False, None, False)
('method', 'method', <class 'fhirclient.models.codeableconcept.CodeableConcept'>, False, None, False)
('quantity', 'quantity', <class 'fhirclient.models.quantity.Quantity'>, False, None, False)


In [28]:
whatsRequired(myCollection)

Nothing is required


None of these are required, but we'll add `bodySite` and `method`. Both are types of `fhirclient.models.codeableconcept.CodeableConcept`. We'll start with `bodySite`, which will be a SNOMED code for buccal space.

In [29]:
myBodySite = cc.CodeableConcept()
for eproperty in myBodySite.elementProperties():
    print(eproperty)

('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('id', 'id', <class 'str'>, False, None, False)
('coding', 'coding', <class 'fhirclient.models.coding.Coding'>, True, None, False)
('text', 'text', <class 'str'>, False, None, False)


In [30]:
whatsRequired(myBodySite)

Nothing is required


We'll use `coding` which is a type of `fhirclient.models.coding.Coding`. This is a `is_list` so it must be put into a list. This makes sense because codeable concepts can have more than one coding. Here we'll create just one coding.

In [31]:
myBodySiteCoding = co.Coding()
for eproperty in myBodySiteCoding.elementProperties():
    print(eproperty)

('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('id', 'id', <class 'str'>, False, None, False)
('code', 'code', <class 'str'>, False, None, False)
('display', 'display', <class 'str'>, False, None, False)
('system', 'system', <class 'str'>, False, None, False)
('userSelected', 'userSelected', <class 'bool'>, False, None, False)
('version', 'version', <class 'str'>, False, None, False)


In [32]:
whatsRequired(myBodySiteCoding)

Nothing is required


Everything is optional. We'll add `system`, `code`, and `display` to `myBodySiteCoding`. Since `myBodySite` is a codeable concept, it can have multiple codings, so we'll put in a list.

In [33]:
myBodySiteCoding.system = 'http://snomed.info/sct'
myBodySiteCoding.code = '261063000'
myBodySiteCoding.display = 'Buccal space'
myBodySite.coding = [myBodySiteCoding] # is_list for coding is True, so it must be put into a list
print(json.dumps(myBodySite.as_json(), indent=4))

{
    "coding": [
        {
            "code": "261063000",
            "display": "Buccal space",
            "system": "http://snomed.info/sct"
        }
    ]
}


Let's repeat this for `method`, which is another `CodeableConcept`, to indicate it is a swab.

In [34]:
myMethod = cc.CodeableConcept()
myMethodCoding = co.Coding()
myMethodCoding.system = 'http://hl7.org/fhir/v2/0488'
myMethodCoding.code = 'SWA'
myMethodCoding.display = 'Swab'
myMethod.coding = [myMethodCoding] # is_list for coding is True, so must be put into a list
print(json.dumps(myMethod.as_json(), indent=4))

{
    "coding": [
        {
            "code": "SWA",
            "display": "Swab",
            "system": "http://hl7.org/fhir/v2/0488"
        }
    ]
}


Now put `myBodySite` and `myMethod` into `myCollection`, and put `myCollection` into `mySpecimen`.

In [35]:
myCollection.bodySite = myBodySite
myCollection.method = myMethod
mySpecimen.collection = myCollection
print(json.dumps(mySpecimen.as_json(), indent=4))

{
    "collection": {
        "bodySite": {
            "coding": [
                {
                    "code": "261063000",
                    "display": "Buccal space",
                    "system": "http://snomed.info/sct"
                }
            ]
        },
        "method": {
            "coding": [
                {
                    "code": "SWA",
                    "display": "Swab",
                    "system": "http://hl7.org/fhir/v2/0488"
                }
            ]
        }
    },
    "subject": {
        "reference": "urn:uuid:d3bc8ddd-ba12-4646-9757-01ca4b2c2192"
    },
    "resourceType": "Specimen"
}


In [36]:
mySpecimen.uuid = uuid.uuid4().urn
print(mySpecimen.uuid)

urn:uuid:01dbb24a-bebb-4fcd-a25c-479f1e739c44


***

### Putting everything into a Bundle

In [37]:
import fhirclient.models.bundle as bun

In [38]:
myBundle = bun.Bundle()
for eproperty in myBundle.elementProperties():
    print(eproperty)

('id', 'id', <class 'str'>, False, None, False)
('implicitRules', 'implicitRules', <class 'str'>, False, None, False)
('language', 'language', <class 'str'>, False, None, False)
('meta', 'meta', <class 'fhirclient.models.meta.Meta'>, False, None, False)
('entry', 'entry', <class 'fhirclient.models.bundle.BundleEntry'>, True, None, False)
('identifier', 'identifier', <class 'fhirclient.models.identifier.Identifier'>, False, None, False)
('link', 'link', <class 'fhirclient.models.bundle.BundleLink'>, True, None, False)
('signature', 'signature', <class 'fhirclient.models.signature.Signature'>, False, None, False)
('total', 'total', <class 'int'>, False, None, False)
('type', 'type', <class 'str'>, False, None, True)


In [39]:
whatsRequired(myBundle)

('type', 'type', <class 'str'>, False, None, True)


In [40]:
myBundle.type = 'transaction'

In [41]:
print(json.dumps(myBundle.as_json(), indent=4))

{
    "type": "transaction",
    "resourceType": "Bundle"
}


In [42]:
pBundleEntry = bun.BundleEntry()

In [43]:
for eproperty in pBundleEntry.elementProperties(): print(eproperty)

('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('id', 'id', <class 'str'>, False, None, False)
('modifierExtension', 'modifierExtension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('fullUrl', 'fullUrl', <class 'str'>, False, None, False)
('link', 'link', <class 'fhirclient.models.bundle.BundleLink'>, True, None, False)
('request', 'request', <class 'fhirclient.models.bundle.BundleEntryRequest'>, False, None, False)
('resource', 'resource', <class 'fhirclient.models.resource.Resource'>, False, None, False)
('response', 'response', <class 'fhirclient.models.bundle.BundleEntryResponse'>, False, None, False)
('search', 'search', <class 'fhirclient.models.bundle.BundleEntrySearch'>, False, None, False)


In [44]:
pBundleEntry.fullUrl = patient.uuid
pBundleEntry.resource = patient

In [45]:
pBundleEntryRequest = bun.BundleEntryRequest()
for eproperty in pBundleEntryRequest.elementProperties(): print(eproperty)

('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('id', 'id', <class 'str'>, False, None, False)
('modifierExtension', 'modifierExtension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('ifMatch', 'ifMatch', <class 'str'>, False, None, False)
('ifModifiedSince', 'ifModifiedSince', <class 'fhirclient.models.fhirdate.FHIRDate'>, False, None, False)
('ifNoneExist', 'ifNoneExist', <class 'str'>, False, None, False)
('ifNoneMatch', 'ifNoneMatch', <class 'str'>, False, None, False)
('method', 'method', <class 'str'>, False, None, True)
('url', 'url', <class 'str'>, False, None, True)


In [46]:
whatsRequired(pBundleEntryRequest)

('method', 'method', <class 'str'>, False, None, True)
('url', 'url', <class 'str'>, False, None, True)


In [47]:
pBundleEntryRequest.method = "POST"
pBundleEntryRequest.url = "Patient"
pBundleEntry.request = pBundleEntryRequest

In [48]:
myBundle.entry = [pBundleEntry]
print(json.dumps(myBundle.as_json(), indent=4))

{
    "entry": [
        {
            "fullUrl": "urn:uuid:d3bc8ddd-ba12-4646-9757-01ca4b2c2192",
            "request": {
                "method": "POST",
                "url": "Patient"
            },
            "resource": {
                "name": [
                    {
                        "family": "Storm",
                        "given": [
                            "Jonathon",
                            "Lowell"
                        ],
                        "text": "Jonathon Lowell Storm",
                        "use": "official"
                    },
                    {
                        "family": "Storm",
                        "given": [
                            "Johnny"
                        ],
                        "use": "nickname"
                    },
                    {
                        "text": "Human Torch",
                        "use": "nickname"
                    }
                ],
                "resourceType": "Pa

#### adding the Specimen

In [49]:
sBundleEntry = bun.BundleEntry()
sBundleEntry.fullUrl = mySpecimen.uuid
sBundleEntry.resource = mySpecimen
sBundleEntryRequest = bun.BundleEntryRequest()
sBundleEntryRequest.method = "POST"
sBundleEntryRequest.url = "Specimen"
sBundleEntry.request = sBundleEntryRequest
myBundle.entry = [pBundleEntry, sBundleEntry]
print(json.dumps(myBundle.as_json(), indent=4))

{
    "entry": [
        {
            "fullUrl": "urn:uuid:d3bc8ddd-ba12-4646-9757-01ca4b2c2192",
            "request": {
                "method": "POST",
                "url": "Patient"
            },
            "resource": {
                "name": [
                    {
                        "family": "Storm",
                        "given": [
                            "Jonathon",
                            "Lowell"
                        ],
                        "text": "Jonathon Lowell Storm",
                        "use": "official"
                    },
                    {
                        "family": "Storm",
                        "given": [
                            "Johnny"
                        ],
                        "use": "nickname"
                    },
                    {
                        "text": "Human Torch",
                        "use": "nickname"
                    }
                ],
                "resourceType": "Pa

---

Now let's try posting it the FHIR server, using the same settings we used in `makePatient.ipynb`

In [50]:
from pprint import pprint
from fhirclient import client
settings = {
        'app_id': 'my_web_app',
        'api_base': 'http://fhirtest.b12x.org/r3'
    }
smart = client.FHIRClient(settings=settings)
pprint(smart.__dict__)

{'_patient': None,
 '_save_func': <function FHIRClient.<lambda> at 0x111367950>,
 'app_id': 'my_web_app',
 'app_secret': None,
 'launch_context': None,
 'launch_token': None,
 'patient_id': None,
 'redirect': None,
 'scope': 'user/*.* patient/*.read openid profile',
 'server': <server.FHIRServer object at 0x11141f2b0>,
 'wants_patient': True}


In [51]:
response = myBundle.create(smart.server)

In [52]:
help(response)

Help on Response in module requests.models object:

class Response(builtins.object)
 |  The :class:`Response <Response>` object, which contains a
 |  server's response to an HTTP request.
 |  
 |  Methods defined here:
 |  
 |  __bool__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if the status code of the response is between
 |      400 and 600 to see if there was a client error or a server error. If
 |      the status code, is between 200 and 400, this will return True. This
 |      is **not** a check to see if the response code is ``200 OK``.
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, *args)
 |  
 |  __getstate__(self)
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |      Allows you to use a response as an iterator.
 |  
 |  __nonzero__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if

In [53]:
response.__dict__

{'_content': b'{\n  "resourceType": "Bundle",\n  "id": "d78a70c9-4660-4202-bdf3-db293628265b",\n  "type": "transaction-response",\n  "link": [\n    {\n      "relation": "self",\n      "url": "http://fhirtest.b12x.org/r3"\n    }\n  ],\n  "entry": [\n    {\n      "response": {\n        "status": "201 Created",\n        "location": "Patient/4418/_history/1",\n        "etag": "1",\n        "lastModified": "2019-01-24T22:39:07.924+00:00"\n      }\n    },\n    {\n      "response": {\n        "status": "201 Created",\n        "location": "Specimen/4419/_history/1",\n        "etag": "1",\n        "lastModified": "2019-01-24T22:39:07.925+00:00"\n      }\n    }\n  ]\n}',
 '_content_consumed': True,
 '_next': None,
 'status_code': 200,
 'headers': {'Content-Encoding': 'gzip', 'Content-Location': 'http://fhirtest.b12x.org/r3/Bundle/d78a70c9-4660-4202-bdf3-db293628265b', 'Content-Type': 'application/fhir+json;charset=UTF-8', 'Date': 'Thu, 24 Jan 2019 22:39:07 GMT', 'Location': 'http://fhirtest.b12x

In [54]:
print(json.dumps(response.json(), indent=4))

{
    "resourceType": "Bundle",
    "id": "d78a70c9-4660-4202-bdf3-db293628265b",
    "type": "transaction-response",
    "link": [
        {
            "relation": "self",
            "url": "http://fhirtest.b12x.org/r3"
        }
    ],
    "entry": [
        {
            "response": {
                "status": "201 Created",
                "location": "Patient/4418/_history/1",
                "etag": "1",
                "lastModified": "2019-01-24T22:39:07.924+00:00"
            }
        },
        {
            "response": {
                "status": "201 Created",
                "location": "Specimen/4419/_history/1",
                "etag": "1",
                "lastModified": "2019-01-24T22:39:07.925+00:00"
            }
        }
    ]
}


In [55]:
print(json.dumps(response.json(), indent=4))

{
    "resourceType": "Bundle",
    "id": "d78a70c9-4660-4202-bdf3-db293628265b",
    "type": "transaction-response",
    "link": [
        {
            "relation": "self",
            "url": "http://fhirtest.b12x.org/r3"
        }
    ],
    "entry": [
        {
            "response": {
                "status": "201 Created",
                "location": "Patient/4418/_history/1",
                "etag": "1",
                "lastModified": "2019-01-24T22:39:07.924+00:00"
            }
        },
        {
            "response": {
                "status": "201 Created",
                "location": "Specimen/4419/_history/1",
                "etag": "1",
                "lastModified": "2019-01-24T22:39:07.925+00:00"
            }
        }
    ]
}


In [56]:
pprint(response.json())

{'entry': [{'response': {'etag': '1',
                         'lastModified': '2019-01-24T22:39:07.924+00:00',
                         'location': 'Patient/4418/_history/1',
                         'status': '201 Created'}},
           {'response': {'etag': '1',
                         'lastModified': '2019-01-24T22:39:07.925+00:00',
                         'location': 'Specimen/4419/_history/1',
                         'status': '201 Created'}}],
 'id': 'd78a70c9-4660-4202-bdf3-db293628265b',
 'link': [{'relation': 'self', 'url': 'http://fhirtest.b12x.org/r3'}],
 'resourceType': 'Bundle',
 'type': 'transaction-response'}


In [57]:
type(response.json())

dict

In [58]:
type(response.json()['entry'])

list

In [59]:
type(response.json()['entry'][0])

dict

In [60]:
for i in response.json()['entry']:
    location = (i['response']['location'])
    # print(location)
    locationList = (location.split('/'))
    endpoint = locationList[0]
    rid = locationList[1]
    print('\n' + endpoint)
    #print(rid)
    if endpoint == 'Patient':
        resource = p.Patient()
    elif endpoint == 'Specimen':
        resource = s.Specimen()
    resource = resource.read(rid, smart.server)
    print(json.dumps(resource.as_json(), indent=4))


Patient
{
    "id": "4418",
    "meta": {
        "lastUpdated": "2019-01-24T22:39:07.924+00:00",
        "versionId": "1"
    },
    "text": {
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\">Jonathon Lowell <b>STORM </b></div><table class=\"hapiPropertyTable\"><tbody></tbody></table></div>",
        "status": "generated"
    },
    "name": [
        {
            "family": "Storm",
            "given": [
                "Jonathon",
                "Lowell"
            ],
            "text": "Jonathon Lowell Storm",
            "use": "official"
        },
        {
            "family": "Storm",
            "given": [
                "Johnny"
            ],
            "use": "nickname"
        },
        {
            "text": "Human Torch",
            "use": "nickname"
        }
    ],
    "resourceType": "Patient"
}

Specimen
{
    "id": "4419",
    "meta": {
        "lastUpdated": "2019-01-24T22:39:07.924+00:00",
        "versionId": "1"
